# Capstone Project


## Load data
Create function to scan the input files and extract the path and target variable for each image.

In [4]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import os
import numpy as np
from glob import glob
import json
# define function to load train, test, and validation datasets
# return tuple with an array of img_paths and an array of target_params
def load_dataset(path):
    img_files = []
    targets = []
    print("processing:"+ path)
    listing = os.walk(path)
    for dir,subdir,files in listing:
        if not subdir:
            print("processing directory:" + dir)
            for file in files:
                if file.endswith('.json'):
                    target = extract_target(os.path.join(dir, file))
                    targets.append(target)
                    img_file = os.path.join(dir, file.replace('json', 'jpg'))
                    img_files.append(img_file)
                        
    targets = np_utils.to_categorical(np.array(targets))
    return np.array(img_files), targets

def extract_target(file_path):
    with open(file_path) as json_data:
        d = json.load(json_data)
        target = d['meta']['clinical']['benign_malignant'] 
        if target == 'benign':
            return 0
        else:
            return 1
    

In [ ]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset('images/train/ISIC-images')
valid_files, valid_targets = load_dataset('images/validation/ISIC-images')
test_files, test_targets = load_dataset('images/test/ISIC-images')


### Exploratory stats
Explore some of the stats of the dataset

In [ ]:
# print statistics about the dataset
print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

In [ ]:
# calculate the number of benign images in the dataset
def count_benign(dataset):
    benign= 0
    for x in dataset:
        if(x[0]):
            benign = benign+1;
    return benign

# calculate the number of malignant images in the dataset
def count_malignant(dataset):
    malignant= 0
    for x in dataset:
        if(x[1]):
            malignant = malignant+1;
    return malignant   

In [ ]:
train_proportions = []
train_proportions.append(count_benign(train_targets)/len(train_targets))
train_proportions.append(count_malignant(train_targets)/len(train_targets))

valid_proportions = []
valid_proportions.append(count_benign(valid_targets)/len(valid_targets))
valid_proportions.append(count_malignant(valid_targets)/len(valid_targets))

test_proportions = []
test_proportions.append(count_benign(test_targets)/len(test_targets))
test_proportions.append(count_malignant(test_targets)/len(test_targets))

print("Proportion of benign to malignant samples in training set is "+ 
      str(train_proportions[0]) + "% to " + str(train_proportions[1]) +"%")

print("Proportion of benign to malignant samples in validation set is "+ 
       str(valid_proportions[0]) + "% to " + str(valid_proportions[1]) +"%")

print("Proportion of benign to malignant samples in test set is "+ 
       str(test_proportions[0]) + "% to " + str(test_proportions[1]) +"%")

In [ ]:
N = 3
benign = (train_proportions[0], valid_proportions[0], test_proportions[0])
malignant = (train_proportions[1], valid_proportions[1], test_proportions[1])

ind = np.arange(3)    # the x locations for the groups
width = 0.75      # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, benign, width)
p2 = plt.bar(ind, malignant, width, bottom=benign)

plt.ylabel('Proportion of samples')
plt.title('Proportion of samples in datasets')
plt.xticks(ind, ('Training set', 'Validation set', 'Testing set'))
plt.yticks(np.arange(0, 1.1, 0.2))
plt.legend((p1[0], p2[0]), ('Benign samples', 'Malignant samples'))

plt.show()

In [ ]:
from PIL import Image

def extract_image_dimentions(dataset_files):
    size_arr = []
    for x in dataset_files:
        jpgfile = Image.open(x)
        h,w = jpgfile.size
        size_arr.append(h*w)
    return size_arr

train_size = extract_image_dimentions(train_files)
valid_size = extract_image_dimentions(valid_files)
test_size = extract_image_dimentions(test_files)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def visualize_hist(data, title, ax):
    n, bins, patches = plt.hist(data, 100)
    plt.xlabel('image_size')
    plt.ylabel('Count')
    plt.title(title)
    plt.show()

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(3, 1, 1, xticks=[], yticks=[])
visualize_hist(train_size, 'Histogram of train image size distribution', ax)
ax = fig.add_subplot(3, 1, 2, xticks=[], yticks=[])
visualize_hist(valid_size, 'Histogram of validation image size distribution', ax)
ax = fig.add_subplot(3, 1, 3, xticks=[], yticks=[])
visualize_hist(test_size, 'Histogram of testing image size distribution', ax)

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def visualize_img(img_path, ax):
    img = cv2.imread(img_path)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
fig = plt.figure(figsize=(20, 10))
for i in range(12):
    ax = fig.add_subplot(3, 4, i + 1, xticks=[], yticks=[])
    visualize_img(train_files[i], ax)

## Initialise initial transfer learning models
Initialise different initial models for transfer learning, and initialise functions to process data

In [2]:
from keras.applications.xception import Xception

# define Xception model
Xception_model_base = Xception(weights='imagenet', include_top=False)


In [ ]:
from keras.applications.resnet50 import ResNet50
# define ResNet50 model
ResNet50_model_base = ResNet50(weights='imagenet', include_top=False)

In [ ]:
from keras.applications.inception_v3 import InceptionV3

# define inception model
Inceptionv3_model_base = InceptionV3(weights='imagenet', include_top=False)


In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm
import concurrent.futures

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

#load the array of file paths, using the given model bottleneck features, if one is not selected just return original.
def paths_to_tensor(img_paths, modelName='none'): 
    if(modelName == 'Xception'):
        #print("Extracting tensors and bottleneck features for Xception")
        list_of_tensors = [Xception_feature_vec(img_path) for img_path in tqdm(img_paths)]
    elif(modelName== 'ResNet'):
        #print("Extracting tensors and bottleneck features for ResNet50")
        list_of_tensors = [ResNet50_feature_vec(img_path) for img_path in tqdm(img_paths)]
    elif(modelName=='Inception'):
        #print("Extracting tensors and bottleneck features for InceptionV3")
        list_of_tensors = [InceptionV3_feature_vec(img_path) for img_path in tqdm(img_paths)]
    else:
        #print('Extracting tensor for images.')
        list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensors)

In [ ]:
def extract_Resnet50(tensor):
    from keras.applications.resnet50 import ResNet50, preprocess_input
    return ResNet50_model_base.predict(preprocess_input(tensor))  

def extract_Xception(tensor):
    from keras.applications.xception import Xception, preprocess_input
    return Xception_model_base.predict(preprocess_input(tensor))

def extract_InceptionV3(tensor):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input
    return Inceptionv3_model_base.predict(preprocess_input(tensor))

In [ ]:
def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(bottleneck_feature))

def ResNet50_feature_vec(img_path):
    # returns feature vector for image located at img_path
    return extract_Resnet50(path_to_tensor(img_path))

def Xception_predict_labels(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_Xception(path_to_tensor(img_path))
    # return label that is predicted by the model
    return np.argmax(Xception_model.predict(bottleneck_feature))

def Xception_feature_vec(img_path):
    # extract bottleneck features
    return extract_Xception(path_to_tensor(img_path))

def InceptionV3_predict_labels(img_path):
    # extract bottleneck features
    bottleneck_feature = extract_InceptionV3(path_to_tensor(img_path))
    # return label that is predicted by the model
    return np.argmax(Xception_model.predict(bottleneck_feature))

def InceptionV3_feature_vec(img_path):
    # extract bottleneck features
    return extract_InceptionV3(path_to_tensor(img_path))

In [ ]:
original_train = paths_to_tensor(train_files).astype('float32')/255
original_validation = paths_to_tensor(valid_files).astype('float32')/255
original_test = paths_to_tensor(test_files).astype('float32')/255
np.save(open('./images/original_train.npy', 'wb'), original_train)
np.save(open('./images/original_validation.npy', 'wb'), original_validation)
np.save(open('./images/original_test.npy', 'wb'), original_test)

np.save(open('./images/train_targets.npy', 'wb'), train_targets)
np.save(open('./images/valid_targets.npy', 'wb'), valid_targets)
np.save(open('./images/test_targets.npy', 'wb'), test_targets)

In [8]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True   

bottleneck_Xception_train = paths_to_tensor(train_files, 'Xception')
bottleneck_Xception_validation = paths_to_tensor(valid_files, 'Xception')
bottleneck_Xception_test = paths_to_tensor(test_files, 'Xception')

np.save(open('./bottleneck_features/bottleneck_Xception_train.npy', 'wb'), bottleneck_Xception_train)
np.save(open('./bottleneck_features/bottleneck_Xception_validation.npy', 'wb'), bottleneck_Xception_validation)
np.save(open('./bottleneck_features/bottleneck_Xception_test.npy', 'wb'), bottleneck_Xception_test)

NameError: name 'paths_to_tensor' is not defined

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  
bottleneck_ResNet50_train = paths_to_tensor(train_files, 'ResNet')
bottleneck_ResNet50_validation = paths_to_tensor(valid_files, 'ResNet')
bottleneck_ResNet50_test = paths_to_tensor(test_files, 'ResNet')
np.save(open('./bottleneck_features/bottleneck_ResNet50_train.npy', 'wb'), bottleneck_ResNet50_train)
np.save(open('./bottleneck_features/bottleneck_ResNet50_validation.npy', 'wb'), bottleneck_ResNet50_validation)
np.save(open('./bottleneck_features/bottleneck_ResNet50_test.npy', 'wb'), bottleneck_ResNet50_test)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  
bottleneck_Inception_train = paths_to_tensor(train_files, 'Inception')
bottleneck_Inception_validation = paths_to_tensor(valid_files, 'Inception')
bottleneck_Inception_test = paths_to_tensor(test_files, 'Inception')
np.save(open('./bottleneck_features/bottleneck_Inception_train.npy', 'wb'), bottleneck_Inception_train)
np.save(open('./bottleneck_features/bottleneck_Inception_validation.npy', 'wb'), bottleneck_Inception_validation)
np.save(open('./bottleneck_features/bottleneck_Inception_test.npy', 'wb'), bottleneck_Inception_test)

## Load pre-processed data
Load the pre-processed data to save us having to keep processing the raw files

In [5]:
train_targets = np.load(open('./images/train_targets.npy','rb'))
valid_targets = np.load(open('./images/valid_targets.npy','rb'))
test_targets = np.load(open('./images/test_targets.npy','rb'))

In [6]:
original_train = np.load(open('./images/original_train.npy','rb'))
original_validation = np.load(open('./images/original_validation.npy','rb'))
original_test = np.load(open('./images/original_test.npy', 'rb'))


In [7]:
bottleneck_Xception_train = np.load(open('./bottleneck_features/bottleneck_Xception_train.npy','rb'))
bottleneck_Xception_validation = np.load(open('./bottleneck_features/bottleneck_Xception_validation.npy','rb'))
bottleneck_Xception_test = np.load(open('./bottleneck_features/bottleneck_Xception_test.npy','rb'))

In [8]:
bottleneck_ResNet50_train = np.load(open('./bottleneck_features/bottleneck_ResNet50_train.npy','rb'))
bottleneck_ResNet50_validation = np.load(open('./bottleneck_features/bottleneck_ResNet50_validation.npy','rb'))
bottleneck_ResNet50_test = np.load(open('./bottleneck_features/bottleneck_ResNet50_test.npy','rb'))


In [9]:
bottleneck_Inception_train = np.load(open('./bottleneck_features/bottleneck_Inception_train.npy','rb'))
bottleneck_Inception_validation = np.load(open('./bottleneck_features/bottleneck_Inception_validation.npy','rb'))
bottleneck_Inception_test = np.load(open('./bottleneck_features/bottleneck_Inception_test.npy','rb'))


## Helpers for training
Some helper code to assist with training

In [10]:
from keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def assessAccuracy(model, features, targets):
    predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in features]
    test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(targets, axis=1))/len(predictions)
    print('Test accuracy: %.4f%%' % test_accuracy)
    
    from sklearn.metrics import f1_score
    print('f1_score:')
    print(f1_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import confusion_matrix
    print('confusion matrix')
    print(confusion_matrix([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import average_precision_score
    print('Precision')
    print(average_precision_score([np.argmax(r) for r in targets], predictions))
    
    from sklearn.metrics import recall_score
    print('Recall')
    print(recall_score([np.argmax(r) for r in targets], predictions))
    

## Xception model
Train a model using bottleneck features from the Xception model.
Then connect the 2 together and try to fine tune layers of the model.

In [11]:
from keras.applications.xception import Xception

# define Xception model
Xception_model_base = Xception(weights='imagenet', include_top=False)

In [12]:
# Xception 4
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


Xception_model = Sequential()
Xception_model.add(Flatten(input_shape=bottleneck_Xception_train.shape[1:]))
Xception_model.add(Dropout(0.8))
Xception_model.add(Dense(2, activation='softmax'))

Xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 200706    
Total params: 200,706.0
Trainable params: 200,706.0
Non-trainable params: 0.0
_________________________________________________________________


In [13]:
# Xception 3
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


Xception_model3 = Sequential()
Xception_model3.add(Flatten(input_shape=bottleneck_Xception_train.shape[1:]))
Xception_model3.add(Dropout(0.7))
Xception_model3.add(Dense(2, activation='softmax'))

Xception_model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 200706    
Total params: 200,706.0
Trainable params: 200,706.0
Non-trainable params: 0.0
_________________________________________________________________


In [ ]:
# Xception
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


Xception_model = Sequential()
Xception_model.add(Flatten(input_shape=bottleneck_Xception_train.shape[1:]))
Xception_model.add(Dropout(0.5))
Xception_model.add(Dense(2, activation='softmax'))

Xception_model.summary()

In [ ]:
# Xception2
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential


Xception_model = Sequential()
Xception_model.add(Flatten(input_shape=bottleneck_Xception_train.shape[1:]))
Xception_model.add(Dropout(0.9))
Xception_model.add(Dense(2, activation='softmax'))

Xception_model.summary()


## Train initial Xception classifier
Train the initial classifier using bottleneck features

In [ ]:

  
### TODO: Compile the model.
Xception_model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=[f1])

In [ ]:
from keras.callbacks import ModelCheckpoint  

### TODO: Train the model.
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception4.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
# 1,2,3 input_class_weight= {0:1., 1:10.}
input_class_weight= {0:1., 1:20.}

Xception_model.fit(bottleneck_Xception_train, train_targets, 
          validation_data=(bottleneck_Xception_validation, valid_targets),
          epochs=200, batch_size=50, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

In [14]:
# 72% accuracy f1-0.37
#Xception_model.load_weights('saved_models/weights.best.Xception.hdf5')
# 56% accuracy f1-0.29
#Xception_model.load_weights('saved_models/weights.best.Xception2.hdf5')
# 67% accuracy f1-0.38
#Xception_model.load_weights('saved_models/weights.best.Xception3.hdf5')
Xception_model3.load_weights('saved_models/weights.best.Xception3.hdf5')



In [ ]:
#Xception4
assessAccuracy(Xception_model, bottleneck_Xception_test, test_targets)

In [15]:
#Xception3
assessAccuracy(Xception_model3, bottleneck_Xception_test, test_targets)

Test accuracy: 67.3333%
f1_score:
0.375796178344
confusion matrix
[[345 136]
 [ 60  59]]
Precision
0.449181210946
Recall
0.495798319328


In [ ]:
#Xception2
assessAccuracy(Xception_model, bottleneck_Xception_test, test_targets)


In [ ]:
#Xception
assessAccuracy(Xception_model, bottleneck_Xception_test, test_targets)


## Fine tune Xception Classifier
Append final classifier onto base Xception model and train with data again.

In [16]:
Xception_model_base.summary()
print(len(Xception_model_base.layers))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864                                          
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128                                          
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0                                            
___________________________________________________________________________________________

In [17]:
from keras.models import Model
from keras import optimizers
from keras.layers import Input

# create the base pre-trained model
input_tensor = Input(shape=original_train[0].shape)
Xception_model_base = Xception(input_tensor=input_tensor, weights='imagenet', include_top=False)
Xception_model3.load_weights('saved_models/weights.best.Xception3.hdf5')

last_layer_to_use_pretrained = 116

# set the first x layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in Xception_model_base.layers[:last_layer_to_use_pretrained]:
    layer.trainable = False

fine_tuned_Xception_model = Model(inputs=Xception_model_base.input, output= Xception_model3(Xception_model_base.output))

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
fine_tuned_Xception_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=[f1])


//anaconda/envs/capstone-project/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  '` call to the Keras 2 API: ' + signature)


In [52]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.final.hdf5', 
                               verbose=1, save_best_only=True)

#weight the importance of the malignant/benign samples so that the model does not become bias to benign which makes up 80% of samples
input_class_weight= {0:1., 1:4.}

fine_tuned_Xception_model.fit(original_train, train_targets, 
          validation_data=(original_validation, valid_targets),
          epochs=100, batch_size=50, class_weight=input_class_weight, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/100
2000/2000 [==============================] - 743s - loss: 12.1830 - f1: 0.8100 - val_loss: 3.2060 - val_f1: 0.8000
Epoch 2/100
2000/2000 [==============================] - 701s - loss: 12.1828 - f1: 0.8100 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 3/100
2000/2000 [==============================] - 699s - loss: 12.1469 - f1: 0.8095 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 4/100
2000/2000 [==============================] - 685s - loss: 12.1581 - f1: 0.8085 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 5/100
2000/2000 [==============================] - 690s - loss: 12.0533 - f1: 0.8100 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 6/100
2000/2000 [==============================] - 684s - loss: 11.7577 - f1: 0.7950 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 7/100
2000/2000 [==============================] - 684s - loss: 11.6705 - f1: 0.7775 - val_loss: 3.3129 - val_f1: 0.7933
Epoch 8/100
2000/2000 [==============================] - 680s - 

2000/2000 [==============================] - 677s - loss: 0.8044 - f1: 0.6545 - val_loss: 0.6338 - val_f1: 0.5467
Epoch 56/100
2000/2000 [==============================] - 676s - loss: 0.8704 - f1: 0.6255 - val_loss: 0.6676 - val_f1: 0.5267
Epoch 57/100
2000/2000 [==============================] - 674s - loss: 0.8567 - f1: 0.6200 - val_loss: 0.6396 - val_f1: 0.5667
Epoch 58/100
2000/2000 [==============================] - 675s - loss: 0.7323 - f1: 0.6990 - val_loss: 0.6041 - val_f1: 0.6333
Epoch 59/100
2000/2000 [==============================] - 683s - loss: 0.8015 - f1: 0.6525 - val_loss: 0.6424 - val_f1: 0.5600
Epoch 60/100
2000/2000 [==============================] - 674s - loss: 0.8008 - f1: 0.6810 - val_loss: 0.6188 - val_f1: 0.5867
Epoch 61/100
2000/2000 [==============================] - 676s - loss: 0.7235 - f1: 0.6915 - val_loss: 0.6014 - val_f1: 0.6333
Epoch 62/100
2000/2000 [==============================] - 673s - loss: 0.7248 - f1: 0.7135 - val_loss: 0.6162 - val_f1: 0.60

In [18]:
fine_tuned_Xception_model.load_weights('saved_models/weights.best.Xception.final.hdf5')

In [19]:
assessAccuracy(fine_tuned_Xception_model, original_test, test_targets)

Test accuracy: 63.3333%
f1_score:
0.411764705882
confusion matrix
[[303 178]
 [ 42  77]]
Precision
0.509509803922
Recall
0.647058823529



### Generate predictions and store for training/validation
to be used for training our random forrest classifier, and test to be used to validate against. Use the initial model as this is not overfit, like the fine tuned model.

In [20]:
train_pred = fine_tuned_Xception_model.predict(original_train)
val_pred = fine_tuned_Xception_model.predict(original_validation)
test_pred = fine_tuned_Xception_model.predict(original_test)

In [21]:
np.save(open('./cnn_output/xception_train.npy', 'wb'), train_pred)
np.save(open('./cnn_output/xception_validation.npy', 'wb'), val_pred)
np.save(open('./cnn_output/xception_test.npy', 'wb'), test_pred)

In [22]:
train_pred_pre = Xception_model3.predict(bottleneck_Xception_train)
val_pred_pre = Xception_model3.predict(bottleneck_Xception_validation)
test_pred_pre = Xception_model3.predict(bottleneck_Xception_test)

In [23]:
np.save(open('./cnn_output/xception_pre_train.npy', 'wb'), train_pred_pre)
np.save(open('./cnn_output/xception_pre_validation.npy', 'wb'), val_pred_pre)
np.save(open('./cnn_output/xception_pre_test.npy', 'wb'), test_pred_pre)